# Thermodynamic Derivatives

## Helmholtz energy derivatives
Thermodynamic derivatives are at the very heart of teqp.  All models are defined in the form $\alpha^r(T, \rho, z)$, where $\rho$ is the molar density, and z are mole fractions.  There are exceptions for models for which the independent variables are in simulation units (Lennard-Jones and its ilk).

Thereofore, to obtain the residual pressure, it is obtained as a derivative:

$$ p^r = \rho R T \left( \rho \left(\frac{\partial \alpha^r}{\partial \rho}\right)_{T}\right)$$

and other residual thermodynamic properties are defined likewise.

We can define the concise derivative

$$ \Lambda^r_{xy} = (1/T)^x(\rho)^y\left(\frac{\partial^{x+y}(\alpha^r)}{\partial (1/T)^x\partial \rho^y}\right)$$

so we can re-write the derivative above as 

$$ p^r = \rho R T \Lambda^r_{01}$$

Similar definitions apply for all the other residual thermodynamic properties

In [1]:
import teqp
import numpy as np

In [2]:
Tc_K = [300]
pc_Pa = [4e6]
acentric = [0.01]
model = teqp.canonical_PR(Tc_K, pc_Pa, acentric)

In [3]:
z = np.array([1.0])
model.get_Ar01(300,300,z)

-0.06836660379313926

And there are additional methods to obtain all the derivatives up to a given order:


In [12]:
model.get_Ar06n(300,300,z) # derivatives 00, 01, 02, ... 06

[-0.06966138343515413,
 -0.06836660379313926,
 0.0025357822532378147,
 -0.00015701162203571184,
 1.6818628788290574e-05,
 -2.2305940927885907e-06,
 3.8259258513417917e-07]

But more derivatives are slower than fewer:

In [13]:
%timeit model.get_Ar01(300,300,z)
%timeit model.get_Ar04n(300,300,z)

1.93 µs ± 32.9 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)
2.13 µs ± 36.7 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


Note: calling overhead is usually on the order of 1 microsecond

## Virial coefficients

Virial coefficients represent the thermodynamics of the interaction of two-, three-, ... bodies interacting with each other.  They can be obtained rigorously if the potential energy surface of interaction is fully known.  In general, such a surface can only be constructed for small rigid molecules.  Many simple thermodynamic models do a poor job of predicting the thermodynamics captured by the virial coefficients.  

The i-th virial coefficient is defined by 
$$
	B_i = \frac{(\alpha^r)^{(i-1)}}{(i-2)!}
$$
with the concise derivative term
$$
	(\alpha^r)^{(i)} = \lim_{\rho \to 0} \left(\frac{\partial^{i}\alpha^r}{\partial \rho^{i}}\right)_{T,\vec{x}}
$$

teqp supports the virial coefficient directly, there is the ``get_B2vir`` for the second virial coefficient:

In [9]:
model.get_B2vir(300, z)

-0.00023661263734465424

And the ``get_Bnvir`` method that allows for the calculation of higher virial coefficients:

In [8]:
model.get_Bnvir(7, 300, z)

{2: -0.0002366126373446542,
 3: 3.001768410777936e-08,
 4: -3.2409760373816355e-12,
 5: 3.9617816466337214e-16,
 6: -4.552923983836698e-20,
 7: 5.3759278511184914e-24}

The ``get_Bnvir`` function was implemented because when doing automatic differentiation, all the intermediate derivatives are also retained.